### Transformers Final Project - Two Towers - BERT Encoder

The notebook is a 6 Layer NN Notebook with GPT Training to do GPT Label Testing.

The purpose of this notebook to do Two Towers with a 6 NN architecture with GPT labeled ESG or not training data to find relevant ESG articles based on a particular industry as each industry has their own unique SASB Query. This notebook is to test out doing a Two Towers Information Retrieval architecture with a BERT encoder

**Note**: The training and testing data use the assumption that **Major and Minor are both 'YES' for ESG and No is 'No' for ESG**.

The notebook uses bert_en_uncased_L-12_H-768_A-12 as the encoder:
- bert: This indicates the model is based on the BERT architecture (Bidirectional Encoder Representations from Transformers), developed by Google. BERT is designed for natural language understanding tasks and is a deep learning model that considers the context from both left and right sides of a token within a sentence.
- en: This specifies the model is trained on English language data.
- uncased: This means the model treats upper and lower case letters as the same. During pre-processing of the data, all letters were converted to lower case.
- L-12: Indicates the model has 12 layers, or transformers blocks. Model depth impacts the model's ability to understand complex language features.
- H-768: The hidden size is 768, which is the number of neurons in each layer. This affects the model's capacity to learn and represent language data.
- A-12: Denotes the model uses 12 attention heads. Attention heads allow the model to focus on different parts of the sentence to better understand the context and the relationships between words.

Additional Files Needed to run the file:
- Train and Test data: 
    - train_df_cleaned_no_stopwords
    - test_set_cleaned_no_stopwords
- Model trained weights (weights trained on GPT labelled data):
    - query_model_weights_v1_transformers.h5
    - candidate_model_weights_v1_transformers.h5
- Results from model for further processing (so to avoid the recomputation of test results and to go directly to success at K information and other metrics):
    - Transformers-TwoTowers-Model_Load_results_df.csv    

In [3]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import matplotlib.pyplot as plt
import re
import ast
import pprint
import numpy as np
from typing import Dict, Text
from sklearn.model_selection import train_test_split
import tensorflow_recommenders as tfrs
from sklearn.metrics.pairwise import cosine_similarity

from pathlib import Path
import spacy
tf.get_logger().setLevel('ERROR')

In [4]:
# Load the specified BERT model
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12'

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')


tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [5]:
# Define the directory path
directory_path = Path(r'C:\Users\tiffa.TIFFANY\OneDrive\Documents\DS 5690 - Transformers\Final Project\Two Tower')


# Define file paths
train_path = directory_path / 'train_df_cleaned_no_stopwords.csv'
test_path = directory_path / 'test_df_cleaned_no_stopwords.csv'


In [6]:
# Read in the cleaned up CSV from other file
df_original = pd.read_csv(train_path, na_filter=False)
df_original.head()

,title_and_content,Ticker,Industry,Company,SASB,GPT_ESG_or_not,GPT_firm_or_not,GPT_sentiment,GPT_topics,ESG_or_not,firm_or_not,human_label_sentiment,url,articleId,title,Concatenated_SASB,lower_title_and_content,lower_Concatenated_SASB,cw_text,cw_sasb_query_text
0,A Bright Spot in Commercial Real Estate: Retai...,CBRE,Real Estate Services,CBRE Group Inc.,{'Sustainability Services': 'In the Real Estat...,Minor,Minor,Neutral,,Minor,Minor,Neutral,https://www.dallasnews.com/business/2023/08/18...,0c2744a7d8ab41f4b81a2dee8b36bb45,"American Airlines sues Skiplagged, claiming ch...",Sustainability Services - In the Real Estate S...,a bright spot in commercial real estate: retai...,sustainability services - in the real estate s...,bright spot commercial real estate retail shop...,sustainability services real estate services i...
1,UPS Trains Non-Union Staff to Deliver Packages...,UPS,Air Freight & Logistics,United Parcel Service Inc B,{'Greenhouse Gas Emissions': 'Air Freight & Lo...,Major,Major,Negative,Labour Practices,Minor,Major,Neutral,https://www.thesun.co.uk/tech/22883701/dangero...,c5810a1c9d26437999ef02bc39eedf1b,Urgent warning over two apps to delete from yo...,Greenhouse Gas Emissions - Air Freight & Logis...,ups trains non-union staff to deliver packages...,greenhouse gas emissions - air freight & logis...,ups trains non union staff deliver packages ca...,greenhouse gas emissions air freight logistics...
2,"A Clean Energy Fund's Challenges, How Twitter ...",STZ,Alcoholic Beverages,Constellation Brands Inc A,{'Water Management': 'Water management include...,Minor,Minor,Positive,"Energy Management, None",Major,Minor,Positive,https://www.theverge.com/2022/12/22/23522535/y...,1bf4ceb04a194274ba6fccbceac0194d,YouTube‚Äôs NFL Sunday Ticket deal is a brilli...,Water Management - Water management includes a...,"a clean energy fund's challenges, how twitter ...",water management - water management includes a...,clean energy fund challenges twitter antics hu...,water management water management includes ent...
3,Live Nation posts 73% jump in revenue and reco...,LYV,Leisure Facilities,Live Nation Entertainment Inc.,{'Customer Safety': 'Leisure facility entities...,Minor,Major,Negative,"Customer Safety, Governance",Minor,Major,Neutral,https://www.ksl.com/article/50656933/ford-reca...,1899eab638d6461796e1d93bb98e4177,Ford recall over discouraging use of seat belts,Customer Safety - Leisure facility entities op...,live nation posts 73% jump in revenue and reco...,customer safety - leisure facility entities op...,live nation posts 73 jump revenue record atten...,customer safety leisure facility entities oper...
4,Corporate Responsibility at T-Mobile: Reaching...,TMUS,Telecommunication Services,T-Mobile US Inc,{'Competitive Behaviour & Open Internet': 'The...,Major,Major,Positive,"Competitive Behaviour & Open Internet, Product...",Major,Major,Positive,https://www.indiatimes.com/technology/news/goo...,5976d28c358745b7a44d50558f3fd4c6,Indian Regulator Says Google's Data Hegemony I...,Competitive Behaviour & Open Internet - The Te...,corporate responsibility at t-mobile: reaching...,competitive behaviour & open internet - the te...,corporate responsibility t mobile reaching new...,competitive behaviour open internet telecommun...


In [7]:
test_df = pd.read_csv(test_path, na_filter=False)

df = df_original
df = df[['cw_text', 'cw_sasb_query_text', 'GPT_ESG_or_not']]
train_df = df

,title_and_content,Ticker,Industry,Company,SASB,GPT_ESG_or_not,GPT_firm_or_not,GPT_sentiment,GPT_topics,ESG_or_not,firm_or_not,human_label_sentiment,url,articleId,title,Concatenated_SASB,lower_title_and_content,lower_Concatenated_SASB,cw_text,cw_sasb_query_text
0,New York Cements Itself as the Gold Mining Cap...,NEM,Metals & Mining,Newmont Corp,{'Tailings Storage Facilities Management': 'Th...,Minor,Major,Positive,"Community Relations, Business Ethics & Transpa...",No,No,No,https://www.newsmax.com/newsmax-tv/fitzgerald-...,c12355d81050473e89f4163372441061,Rep. Fitzgerald to Newsmax: DirecTV Dropping N...,Tailings Storage Facilities Management - The M...,new york cements itself as the gold mining cap...,tailings storage facilities management - the m...,new york cements gold mining capital world new...,tailings storage facilities management metals ...
1,"Shareholders v. Tesla, Nasdaq's diversity rule...",NDAQ,Security & Commodity Exchanges,Nasdaq Inc,{'Managing Conflicts of Interest': 'Security a...,Major,Major,Negative,"Managing Conflicts of Interest, Promoting Tran...",Major,Major,Positive,https://www.axios.com/pro/media-deals/2023/05/...,fcbd16768c584451912d7121a259ad9d,YouTube praises AI transformation at Brandcast,Managing Conflicts of Interest - Security and ...,"shareholders v. tesla, nasdaq's diversity rule...",managing conflicts of interest - security and ...,shareholders v. tesla nasdaq diversity rule se...,managing conflicts interest security commodity...
2,"FedEx closing more locations, planning to furl...",FDX,Air Freight & Logistics,FedEx Corp,{'Greenhouse Gas Emissions': 'Air Freight & Lo...,Minor,Major,Negative,"Employee Health & Safety, Labour Practices, Su...",Minor,Major,Negative,https://www.theguardian.com/technology/2023/ju...,3cb0ea7cb1cb40608c1cfc1e172ebc3e,Nick Clegg defends release of open-source AI m...,Greenhouse Gas Emissions - Air Freight & Logis...,"fedex closing more locations, planning to furl...",greenhouse gas emissions - air freight & logis...,fedex closing locations planning furlough empl...,greenhouse gas emissions air freight logistics...
3,Modelo Maker Profits From Bud Light‚Äö√Ñ√¥s De...,STZ,Alcoholic Beverages,Constellation Brands Inc A,{'Water Management': 'Water management include...,Minor,Minor,Positive,"Water Management, Packaging Lifecycle Manageme...",No,No,No,https://www.washingtonexaminer.com/restoring-a...,7b188eebdd7c42ed9ca51237d0989674,Conservative group targets Bank of America in ...,Water Management - Water management includes a...,modelo maker profits from bud light‚äö√ñ√¥s de...,water management - water management includes a...,modelo maker profits bud light‚äö√ñ√¥s decline...,water management water management includes ent...
4,Med tech investors paying up for patents - Med...,ILMN,Medical Equipment & Supplies,Illumina Inc,{'Product Safety': 'Information on product saf...,Minor,Major,Negative,Business Ethics,No,No,No,https://www.cleveland.com/business/2023/01/goo...,14b0ee5d771844c7838718faf0905545,"Google slashes 12,000 jobs to cope with shrink...",Product Safety - Information on product safety...,med tech investors paying up for patents - med...,product safety - information on product safety...,med tech investors paying patents med tech sta...,product safety information product safety effe...


In [8]:
# We only want to train on if it is ESG relevant news or not so this is following the assumption that Major and Minor are ESG
train_df = train_df[train_df['GPT_ESG_or_not'] != 'No'].reset_index(drop=True)

test_df = test_df[['cw_text', 'cw_sasb_query_text', 'GPT_ESG_or_not', 'Industry' , 'ESG_or_not']]

In [9]:
train_df

,cw_text,cw_sasb_query_text,GPT_ESG_or_not
0,bright spot commercial real estate retail shop...,sustainability services real estate services i...,Minor
1,ups trains non union staff deliver packages ca...,greenhouse gas emissions air freight logistics...,Major
2,clean energy fund challenges twitter antics hu...,water management water management includes ent...,Minor
3,live nation posts 73 jump revenue record atten...,customer safety leisure facility entities oper...,Minor
4,corporate responsibility t mobile reaching new...,competitive behaviour open internet telecommun...,Major
...,...,...,...
3383,ups employees arrested alleged cocaine traffic...,greenhouse gas emissions air freight logistics...,Minor
3384,western officials visit uae efforts halt expor...,recruiting managing global skilled workforce e...,Major
3385,tyson ceo says company got hit mouth missing q...,land use ecological impacts meat poultry dairy...,Minor
3386,investors implore government step silicon vall...,factors credit analysis financial intermediari...,Major


In [10]:
test_df

,cw_text,cw_sasb_query_text,GPT_ESG_or_not,Industry,ESG_or_not
0,new york cements gold mining capital world new...,tailings storage facilities management metals ...,Minor,Metals & Mining,No
1,shareholders v. tesla nasdaq diversity rule se...,managing conflicts interest security commodity...,Major,Security & Commodity Exchanges,Major
2,fedex closing locations planning furlough empl...,greenhouse gas emissions air freight logistics...,Minor,Air Freight & Logistics,Minor
3,modelo maker profits bud light‚äö√ñ√¥s decline...,water management water management includes ent...,Minor,Alcoholic Beverages,No
4,med tech investors paying patents med tech sta...,product safety information product safety effe...,Minor,Medical Equipment & Supplies,No
...,...,...,...,...,...
1036,lockheed martin stumbles supply chain wsj dema...,product safety product safety important consid...,Major,Aerospace & Defence,Major
1037,banks rush borrow record breaking $ 165 billio...,factors credit analysis financial intermediari...,Major,Commercial Banks,Major
1038,ohio train derailment norfolk southern ceo say...,greenhouse gas emissions rail transportation i...,Major,Rail Transportation,Major
1039,at&t verizon t mobile avoid $ 200 million fine...,competitive behaviour open internet telecommun...,Major,Telecommunication Services,Major


## Create the 6 layer NN Two Towers Architecture

We gave query and candidate towers the same architecture. We conducted hyperparameter fine-tuning on the number of layers, the final units, activation functions, and number of epochs.

In [11]:
def query_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='qtext')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='qpreprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='qBERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(15, activation="relu", name='qclassifier_layer1')(net)
    net = tf.keras.layers.Dense(10, activation="relu", name='qclassifier_layer2')(net)
    net = tf.keras.layers.Dense(5, activation="sigmoid", name='qclassifier')(net)
    return tf.keras.Model(text_input, net)

def candidate_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='ctext')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='cpreprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='cBERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(15, activation="relu", name='cclassifier_layer1')(net)
    net = tf.keras.layers.Dense(10, activation="relu", name='cclassifier_layer2')(net)
    net = tf.keras.layers.Dense(5, activation="sigmoid", name='cclassifier')(net)
    return tf.keras.Model(text_input, net)


candidate_model = candidate_classifier_model()
query_model = query_classifier_model()

# Load the saved weights to prevent having to retrain each time
query_model.load_weights('query_model_weights_v1_transformers.h5')
candidate_model.load_weights('candidate_model_weights_v1_transformers.h5')

In [12]:
# Define the information retrieval model
class TwoTowerRetrievalModel(tfrs.Model):
    def __init__(self, query_model, candidate_model):
        super().__init__()
        self.query_model = query_model
        self.candidate_model = candidate_model
        self.task = tfrs.tasks.Retrieval()

    def compute_loss(self, features, training=True):
        query_embeddings = self.query_model(features[1])
        candidate_embeddings = self.candidate_model(features[0])
        return self.task(query_embeddings, candidate_embeddings)
    
metrics = tf.metrics.CosineSimilarity(axis=1)  
retrieval_model = TwoTowerRetrievalModel(query_model, candidate_model)
retrieval_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.01),
                        loss=tf.keras.losses.CosineSimilarity(axis=1), metrics = metrics)

# # Comment out fit when/if reloading the model weights from above cell
# retrieval_model.fit(x = train_df.loc[:, 'cw_text': 'cw_sasb_query_text'], epochs = 3)

Epoch 1/3
106/106 [==============================] - 606s 6s/step - loss: 1.3819 - regularization_loss: 0.0000e+00 - total_loss: 1.3819
Epoch 2/3
106/106 [==============================] - 574s 5s/step - loss: 1.3759 - regularization_loss: 0.0000e+00 - total_loss: 1.3759
Epoch 3/3
106/106 [==============================] - 571s 5s/step - loss: 1.3816 - regularization_loss: 0.0000e+00 - total_loss: 1.3816


In [13]:
# Evaluate the model
x_val = retrieval_model.evaluate(test_df.loc[:, 'cw_text': 'cw_sasb_query_text'])
print(f"Retrieval model performance on test set: {x_val}")

# Save model weights on the ~4000 training data points if conducting the training for the first time
# query_model.save_weights('query_model_weights_v1_transformers.h5')
# candidate_model.save_weights('candidate_model_weights_v1_transformers.h5')

33/33 [==============================] - 52s 2s/step - loss: 1.3855 - regularization_loss: 0.0000e+00 - total_loss: 1.3855
Retrieval model performance on test set: [1.3867532014846802, 0, 1.3867532014846802]


In [11]:
test_df_copy = test_df.copy()

# Remove "NA" and get unique queries and candidates
unique_queries = test_df_copy.loc[test_df_copy['cw_sasb_query_text'] != "NA", ['cw_sasb_query_text', 'Industry']].drop_duplicates().reset_index(drop=True)

# Get unique candidates
unique_candidates = test_df_copy['cw_text'].unique()

# Get unique queries after removing potential unclean duplicates to get 61 industries
unique_queries = unique_queries.groupby('Industry')['cw_sasb_query_text'].first().reset_index()

In [13]:
unique_queries

,Industry,cw_sasb_query_text
0,Advertising & Marketing,advertising integrity entities legal responsib...
1,Aerospace & Defence,product safety product safety important consid...
2,Agricultural Products,greenhouse gas emissions entities agricultural...
3,Air Freight & Logistics,greenhouse gas emissions air freight logistics...
4,Airlines,competitive behaviour airlines industry charac...
...,...,...
56,Solar Technology & Project Developers,hazardous waste management solar panel manufac...
57,Telecommunication Services,competitive behaviour open internet telecommun...
58,Tobacco,marketing practices tobacco product labelling ...
59,Toys & Sporting Goods,chemical safety hazards products consumers reg...


In [18]:
# Pre-compute query embeddings
query_embeddings = {query_row['cw_sasb_query_text']: query_model.predict(np.array([query_row['cw_sasb_query_text']]).reshape(-1, 1)) for _, query_row in unique_queries.iterrows()}

# Pre-compute candidate embeddings
candidate_embeddings = {candidate_row['cw_text']: candidate_model.predict(np.array([candidate_row['cw_text']]).reshape(-1, 1)) for _, candidate_row in test_df_copy.iterrows()}

# Get the respective cosine similarity value - we are assuming industry is a known label to reduce pair-wise compuation size
results = []
for _, query_row in unique_queries.iterrows():
    query_industry = query_row['Industry']
    query_extracted_paragraph = query_row['cw_sasb_query_text']
    query_embedding = query_embeddings[query_extracted_paragraph]

    for _, candidate_row in test_df_copy[test_df_copy['Industry'] == query_industry].iterrows():
        title_and_content = candidate_row['cw_text']
        candidate_embedding = candidate_embeddings[title_and_content]

        similarity_score = cosine_similarity(query_embedding, candidate_embedding)[0][0]

        results.append({
            'cw_sasb_query_text': query_extracted_paragraph,
            'cw_text': title_and_content,
            'similarity_score': similarity_score
        })

results_df = pd.DataFrame(results)


1/1 [==============================] - 0s 391ms/step


1/1 [==============================] - 0s 240ms/step


1/1 [==============================] - 0s 297ms/step


1/1 [==============================] - 0s 295ms/step


In [38]:
# Taking a look at a random sample of data
results_df.sample(5)

,cw_sasb_query_text,cw_text,similarity_score
598,employee diversity inclusion investment bankin...,surveillance jpmorgan reaps boon crisis happen...,0.892575
279,factors credit analysis financial intermediari...,banking turmoil crisis downside risks real iif...,0.900291
615,employee diversity inclusion investment bankin...,dc insider jamie dimon poised benefit banking ...,0.892707
804,management legal regulatory environment oil ga...,halliburton company hal attracting investor at...,0.899396
899,sustainability services real estate services i...,amid slow return work recession fears companie...,0.899988


In [20]:
# Save the dataframe for future success at K reload if running the test data for the first time
# results_df.to_csv("Transformers-TwoTowers-Model_Load_results_df.csv", index=False)

In [23]:
# Confirm on results size is as expected
results_df.shape

(1041, 3)

In [15]:
# Uncomment to read in csv if loading in success at K
# results_df = pd.read_csv('Transformers-TwoTowers-Model_Load_results_df.csv', na_filter=False)

Bring back the missing columns like industry to do the grouping

In [16]:
merged_df = pd.merge(results_df, unique_queries[['cw_sasb_query_text', 'Industry']], 
                     on='cw_sasb_query_text', how='left')

merged_df

,cw_sasb_query_text,cw_text,similarity_score,Industry
0,advertising integrity entities legal responsib...,interpublic group ipg stock sinks market gains...,0.900015,Advertising & Marketing
1,advertising integrity entities legal responsib...,interpublic group ipg outpaces stock market ga...,0.900175,Advertising & Marketing
2,advertising integrity entities legal responsib...,credera launches global cross functional ai co...,0.899739,Advertising & Marketing
3,advertising integrity entities legal responsib...,ddb chicago promotes kiska howell evp head bus...,0.900139,Advertising & Marketing
4,advertising integrity entities legal responsib...,ad firm suggests äö√ñ√≤pause‚äö√ñ√¥ twitter ad...,0.899620,Advertising & Marketing
...,...,...,...,...
1036,greenhouse gas emissions landfills significant...,recycling collection changing horsham township...,0.900493,Waste Management
1037,greenhouse gas emissions landfills significant...,waste management nyse wm returns hit wall sure...,0.901158,Waste Management
1038,greenhouse gas emissions landfills significant...,waste management nyse wm shareholders earned 1...,0.902090,Waste Management
1039,greenhouse gas emissions landfills significant...,waste management inc wm stock forecasts growth...,0.901579,Waste Management


In [17]:
# Count the number of null values in each column of merged_df
null_counts = merged_df.isnull().sum()

# Display the null counts
print(null_counts)

cw_sasb_query_text    0
cw_text               0
similarity_score      0
Industry              0
dtype: int64


In [18]:
test_df_original = pd.read_csv(test_path, na_filter=False)

test_df_original.head()

,title_and_content,Ticker,Industry,Company,SASB,GPT_ESG_or_not,GPT_firm_or_not,GPT_sentiment,GPT_topics,ESG_or_not,firm_or_not,human_label_sentiment,url,articleId,title,Concatenated_SASB,lower_title_and_content,lower_Concatenated_SASB,cw_text,cw_sasb_query_text
0,New York Cements Itself as the Gold Mining Cap...,NEM,Metals & Mining,Newmont Corp,{'Tailings Storage Facilities Management': 'Th...,Minor,Major,Positive,"Community Relations, Business Ethics & Transpa...",No,No,No,https://www.newsmax.com/newsmax-tv/fitzgerald-...,c12355d81050473e89f4163372441061,Rep. Fitzgerald to Newsmax: DirecTV Dropping N...,Tailings Storage Facilities Management - The M...,new york cements itself as the gold mining cap...,tailings storage facilities management - the m...,new york cements gold mining capital world new...,tailings storage facilities management metals ...
1,"Shareholders v. Tesla, Nasdaq's diversity rule...",NDAQ,Security & Commodity Exchanges,Nasdaq Inc,{'Managing Conflicts of Interest': 'Security a...,Major,Major,Negative,"Managing Conflicts of Interest, Promoting Tran...",Major,Major,Positive,https://www.axios.com/pro/media-deals/2023/05/...,fcbd16768c584451912d7121a259ad9d,YouTube praises AI transformation at Brandcast,Managing Conflicts of Interest - Security and ...,"shareholders v. tesla, nasdaq's diversity rule...",managing conflicts of interest - security and ...,shareholders v. tesla nasdaq diversity rule se...,managing conflicts interest security commodity...
2,"FedEx closing more locations, planning to furl...",FDX,Air Freight & Logistics,FedEx Corp,{'Greenhouse Gas Emissions': 'Air Freight & Lo...,Minor,Major,Negative,"Employee Health & Safety, Labour Practices, Su...",Minor,Major,Negative,https://www.theguardian.com/technology/2023/ju...,3cb0ea7cb1cb40608c1cfc1e172ebc3e,Nick Clegg defends release of open-source AI m...,Greenhouse Gas Emissions - Air Freight & Logis...,"fedex closing more locations, planning to furl...",greenhouse gas emissions - air freight & logis...,fedex closing locations planning furlough empl...,greenhouse gas emissions air freight logistics...
3,Modelo Maker Profits From Bud Light‚Äö√Ñ√¥s De...,STZ,Alcoholic Beverages,Constellation Brands Inc A,{'Water Management': 'Water management include...,Minor,Minor,Positive,"Water Management, Packaging Lifecycle Manageme...",No,No,No,https://www.washingtonexaminer.com/restoring-a...,7b188eebdd7c42ed9ca51237d0989674,Conservative group targets Bank of America in ...,Water Management - Water management includes a...,modelo maker profits from bud light‚äö√ñ√¥s de...,water management - water management includes a...,modelo maker profits bud light‚äö√ñ√¥s decline...,water management water management includes ent...
4,Med tech investors paying up for patents - Med...,ILMN,Medical Equipment & Supplies,Illumina Inc,{'Product Safety': 'Information on product saf...,Minor,Major,Negative,Business Ethics,No,No,No,https://www.cleveland.com/business/2023/01/goo...,14b0ee5d771844c7838718faf0905545,"Google slashes 12,000 jobs to cope with shrink...",Product Safety - Information on product safety...,med tech investors paying up for patents - med...,product safety - information on product safety...,med tech investors paying patents med tech sta...,product safety information product safety effe...


In [19]:
final_results_df = pd.merge(merged_df, test_df_original[['cw_text', 'url', 'articleId', 'title']], 
                    on='cw_text', how='left')
final_results_df.head()

,cw_sasb_query_text,cw_text,similarity_score,Industry,url,articleId,title
0,advertising integrity entities legal responsib...,interpublic group ipg stock sinks market gains...,0.900015,Advertising & Marketing,https://www.usatoday.com/story/news/politics/2...,b33dfc62fefc44608c7f3bbdf62de5af,Trump calls on Rupert Murdoch to back false 20...
1,advertising integrity entities legal responsib...,interpublic group ipg outpaces stock market ga...,0.900175,Advertising & Marketing,https://www.cbsnews.com/sanfrancisco/news/man-...,e0786d05237043888004cb1822b45fd6,Man killed by runaway U-Haul truck in San Fran...
2,advertising integrity entities legal responsib...,credera launches global cross functional ai co...,0.899739,Advertising & Marketing,https://www.yahoo.com/lifestyle/best-tech-deal...,7bd513891e414c1aa4d130a3ec80ed44,Apple AirPods for $99 (that's $30 off) ‚Äî plu...
3,advertising integrity entities legal responsib...,ddb chicago promotes kiska howell evp head bus...,0.900139,Advertising & Marketing,https://www.businessinsider.com/private-credit...,6ad4e5a8266a4d82a5888e8c0a75967e,"How 5 private-credit players, from Apollo to C..."
4,advertising integrity entities legal responsib...,ad firm suggests äö√ñ√≤pause‚äö√ñ√¥ twitter ad...,0.899620,Advertising & Marketing,https://www.westernjournal.com/democratic-sena...,3a86ecf280094c1aad903739b3aab64d,Democratic Senator Discussed Social Media Cens...


In [20]:
# Count the number of null values in each column of final_results_df
null_counts = final_results_df.isnull().sum()

# Display the null counts
print(null_counts)

cw_sasb_query_text    0
cw_text               0
similarity_score      0
Industry              0
url                   0
articleId             0
title                 0
dtype: int64


In [21]:
# Check final_results_df
final_results_df

,cw_sasb_query_text,cw_text,similarity_score,Industry,url,articleId,title
0,advertising integrity entities legal responsib...,interpublic group ipg stock sinks market gains...,0.900015,Advertising & Marketing,https://www.usatoday.com/story/news/politics/2...,b33dfc62fefc44608c7f3bbdf62de5af,Trump calls on Rupert Murdoch to back false 20...
1,advertising integrity entities legal responsib...,interpublic group ipg outpaces stock market ga...,0.900175,Advertising & Marketing,https://www.cbsnews.com/sanfrancisco/news/man-...,e0786d05237043888004cb1822b45fd6,Man killed by runaway U-Haul truck in San Fran...
2,advertising integrity entities legal responsib...,credera launches global cross functional ai co...,0.899739,Advertising & Marketing,https://www.yahoo.com/lifestyle/best-tech-deal...,7bd513891e414c1aa4d130a3ec80ed44,Apple AirPods for $99 (that's $30 off) ‚Äî plu...
3,advertising integrity entities legal responsib...,ddb chicago promotes kiska howell evp head bus...,0.900139,Advertising & Marketing,https://www.businessinsider.com/private-credit...,6ad4e5a8266a4d82a5888e8c0a75967e,"How 5 private-credit players, from Apollo to C..."
4,advertising integrity entities legal responsib...,ad firm suggests äö√ñ√≤pause‚äö√ñ√¥ twitter ad...,0.899620,Advertising & Marketing,https://www.westernjournal.com/democratic-sena...,3a86ecf280094c1aad903739b3aab64d,Democratic Senator Discussed Social Media Cens...
...,...,...,...,...,...,...,...
1052,greenhouse gas emissions landfills significant...,recycling collection changing horsham township...,0.900493,Waste Management,https://patch.com/massachusetts/waltham/two-wa...,f6bc301c26d147cfacebb5df4f930816,"Waltham-Based Thermo Fisher, Global Partners M..."
1053,greenhouse gas emissions landfills significant...,waste management nyse wm returns hit wall sure...,0.901158,Waste Management,https://www.forbes.com/sites/ewanspence/2023/0...,9e2097e08ae44b268401bab02901f69c,"Android Circuit: Galaxy S24 Leak, OnePlus Pad ..."
1054,greenhouse gas emissions landfills significant...,waste management nyse wm shareholders earned 1...,0.902090,Waste Management,https://finance.yahoo.com/news/microsoft-bent-...,59a56953b8cc4e24a7d8e7f12bc1550f,Microsoft Bent On Crushing Opposition To Activ...
1055,greenhouse gas emissions landfills significant...,waste management inc wm stock forecasts growth...,0.901579,Waste Management,https://www.cnbc.com/2022/10/28/analysts-remai...,0703f9a0e4234482a7e8ba2fe7edcdef,Analysts remain confident in Amazon long term ...


In [22]:
# Check what is the min and max value of the cosine similarity score for final_results_df

min_value = final_results_df['similarity_score'].min()
max_value = final_results_df['similarity_score'].max()

print(f"Minimum value in 'similarity_score': {min_value}")
print(f"Maximum value in 'similarity_score': {max_value}")

Minimum value in 'similarity_score': 0.88996536
Maximum value in 'similarity_score': 0.9038078


## Calculating Model Comparison Metrics

In this section after doing some model prep work, we are computing the following metrics:
* Success at K - A metric to establish whether we get a hit/relevant ESG article within K. Measures whether the relevant document (or item) appears in the top K positions of the model's ranking.
* Mean Reciprocal Rank (MRR) - MRR provides insight into the model's ability to return relevant items at higher ranks. It measures when does the first relevant ESG article appears. The closer this final number is to 1, the better the system is at giving you the right answers upfront.  
* Precision at K - Measures the proportion of retrieved documents that are relevant among the top K documents retrieved. It's calculated by dividing the number of relevant documents in the top K by K.
* Recall at K - Measures the proportion of relevant documents retrieved in the top K positions out of all relevant documents available. 
* F1 Score at K - Combines precision and recall into a single metric, offering a more comprehensive evaluation of the model's performance. It helps balance the trade-off between precision and recall, ensuring that neither is disproportionately favored.

**Note**:This is the section user should have already reloaded results_df and applied the final dataframe changes to get final_results_df before calculating the metrics. Also, in this section, we will do the mapping to have the testing data use the assumption that **Major and Minor are both 'YES' for ESG and No is 'No' for ESG**.

In [23]:
# Ensure you have test_df_copy reloaded from above
test_df_copy

,cw_text,cw_sasb_query_text,GPT_ESG_or_not,Industry,ESG_or_not
0,new york cements gold mining capital world new...,tailings storage facilities management metals ...,Minor,Metals & Mining,No
1,shareholders v. tesla nasdaq diversity rule se...,managing conflicts interest security commodity...,Major,Security & Commodity Exchanges,Major
2,fedex closing locations planning furlough empl...,greenhouse gas emissions air freight logistics...,Minor,Air Freight & Logistics,Minor
3,modelo maker profits bud light‚äö√ñ√¥s decline...,water management water management includes ent...,Minor,Alcoholic Beverages,No
4,med tech investors paying patents med tech sta...,product safety information product safety effe...,Minor,Medical Equipment & Supplies,No
...,...,...,...,...,...
1036,lockheed martin stumbles supply chain wsj dema...,product safety product safety important consid...,Major,Aerospace & Defence,Major
1037,banks rush borrow record breaking $ 165 billio...,factors credit analysis financial intermediari...,Major,Commercial Banks,Major
1038,ohio train derailment norfolk southern ceo say...,greenhouse gas emissions rail transportation i...,Major,Rail Transportation,Major
1039,at&t verizon t mobile avoid $ 200 million fine...,competitive behaviour open internet telecommun...,Major,Telecommunication Services,Major


In [ ]:
# Preparing data to do Success at K
# Sort articles by cosine similarity score for each Industry group
top_sorted_df = final_results_df.groupby('Industry', group_keys=False) \
                  .apply(lambda x: x.sort_values('similarity_score', ascending=False))

top_sorted_df = top_sorted_df.reset_index(drop=True)

test_df_relevant = test_df_copy[['cw_text', 'GPT_ESG_or_not']].drop_duplicates()
merged_df_final = pd.merge(top_sorted_df, test_df_relevant, on='cw_text', how='left')

# Mapping - applying the Minor and Major as Yes assumption
mapping = {'Minor': 'Yes', 'Major': 'Yes', 'No': 'No'}

merged_df_final['GPT_ESG_or_not'] = merged_df_final['GPT_ESG_or_not'].map(mapping)

# Adding in the ground truth labels and checking if it looks correct
merged_df_final

In [24]:
# Get Success at K Metrics
# Success at k is about within the top k results, is there at least one relevant item?
def calculate_success_at_k(merged_df, k):
    # Group by 'Industry'
    grouped_df = merged_df.groupby('Industry')
    group_sizes = grouped_df.size()
    hit_count = 0
    total_groups = len(grouped_df)

    for name, group in grouped_df:
        if 'Yes' in group.head(k)['GPT_ESG_or_not'].values:
            hit_count += 1

    hit_rate = hit_count / total_groups
    return hit_rate

# Initialize an empty DataFrame to store results
success_k = pd.DataFrame(columns=['k', 'hit_rate'])

# Create an empty list to store intermediate results
results = []

# Loop through k values from 1 to 5 as we don't expect going pass 5 is necessary
for k in range(1, 6):
    hit_rate = calculate_success_at_k(merged_df_final, k)
    # Store the result as a dictionary in the list
    results.append({'k': k, 'hit_rate': hit_rate})

# Convert the list of dictionaries to a DataFrame
success_k = pd.concat([pd.DataFrame([result]) for result in results], ignore_index=True)

# Display the results
print(success_k)

   k  hit_rate
0  1  0.688525
1  2  0.836066
2  3  0.950820
3  4  0.967213
4  5  1.000000


In [25]:
# Calculate MRR
# MRR is how well does the model rank the first relevant item, on average, across all queries a k 
# value does not play a role here.
def calculate_mrr(merged_df):
    # Group by 'Industry' to process each query group separately
    grouped_df = merged_df.groupby('Industry')
    total_queries = len(grouped_df)  # Total number of queries
    sum_reciprocal_rank = 0  # Initialize the sum of reciprocal ranks
    
    for name, group in grouped_df:
        # Sort each group just in case it's not sorted by relevance (similarity score)
        group = group.sort_values('similarity_score', ascending=False)
        # Find the index (rank) of the first 'Yes' in the sorted group
        first_relevant_index = group['GPT_ESG_or_not'].eq('Yes').idxmax()
        if group.loc[first_relevant_index, 'GPT_ESG_or_not'] == 'Yes':
            rank = group.index.get_loc(first_relevant_index) + 1  # Get rank (1-based)
            sum_reciprocal_rank += 1 / rank  # Add the reciprocal of the rank to the sum
    
    # Calculate the mean of the reciprocal ranks
    mrr = sum_reciprocal_rank / total_queries  
    return mrr

mrr_score = calculate_mrr(merged_df_final)
print(f"The Mean Reciprocal Rank (MRR) is: {mrr_score}")


The Mean Reciprocal Rank (MRR) is: 0.8112021857923498


In [32]:
# Calculate the precision, recall, and f1 at K
# We set k at what we decided from Success at K
def calculate_precision_recall_at_k_per_query(group, k):
    # Convert 'Yes'/'No' in 'GPT_ESG_or_not' to 1/0 for calculation
    group['is_correct'] = group['GPT_ESG_or_not'].apply(lambda x: 1 if x == 'Yes' else 0)

    # Sort the group by similarity_score in descending order and take top K
    top_k = group.sort_values('similarity_score', ascending=False).head(k)

    # Calculate how many of the top K are correct
    correct_in_top_k = top_k['is_correct'].sum()

    # Calculate Precision at K
    precision_at_k = correct_in_top_k / k if k > 0 else 0

    # Calculate Recall at K
    total_relevant_in_group = group['is_correct'].sum()
    recall_at_k = correct_in_top_k / total_relevant_in_group if total_relevant_in_group > 0 else 0
    
    # Calculate F1 at K
    if precision_at_k + recall_at_k > 0:
        f1_at_k = 2 * (precision_at_k * recall_at_k) / (precision_at_k + recall_at_k)
    else:
        f1_at_k = 0

    return precision_at_k, recall_at_k, f1_at_k

# Apply the function to each group and calculate the mean Precision, Recall, and F1 at K
# Use merged_df_final if want to see all of the initial test run results
results = merged_df_final.groupby('Industry').apply(calculate_precision_recall_at_k_per_query, k=3)

# To see the overall average Precision, Recall, and F1 at K
average_precision_at_k = results.map(lambda x: x[0]).mean()
average_recall_at_k = results.map(lambda x: x[1]).mean()
average_f1_at_k = results.map(lambda x: x[2]).mean()

print(f"Average Precision at K: {average_precision_at_k}")
print(f"Average Recall at K: {average_recall_at_k}")
print(f"Average F1 at K: {average_f1_at_k}")

Average Precision at K: 0.7158469945355189
Average Recall at K: 0.21535806001358176
Average F1 at K: 0.30110705755499684
